# Train custom nuclei segmentation model

In the following, we describe how to train our proposed nuclei segmentation model with your custom data using fluospotter.

First, we need to install fluospotter if not installed:

`pip install fluospotter`

Or alternatively:

`pip install https://github.com/arnaublanco/Fluospotter`

Secondly, we need to load your data, we can do so by placing the data as TIF files with format (depth, height, width), that is, a stack of 2D images, in the folders called `train`, `valid` and `test`.

&nbsp;
<img src="images/test_folder.png" width=700>
&nbsp;

We need to ensure that the ratio train/validation/test is approximately 75/15/10. For example, if I had 20 volumes, the ratio would be 15 volumes for training, 3 volumes for validation and 2 for testing, as shown in the following image.

&nbsp;
<img src="images/test_folder_data.png" width=700>
&nbsp;

Nonetheless, the ratio will ultimately depend on you, but this is a fair starting point. Similarly, we need another folder with the masks/labels containing the exact same three folders (e.g. `labels`). Therefore:

In [ ]:
from fluospotter.datasets import Dataset

data = Dataset(data_dir="data", segmentation_dir="labels", training=True) # Here we specify the root folders, which in our case are `data` and `labels` that contains the three folders

It's important that that the data in `data` and `labels` have the same name so that fluospotter can match them. For example, `Location-03.tif` in `data/test` will be matched with `Location-03.tif` in `labels/test`. If it can't be matched, an error will be raised.

Now, we need to create an instance of a model, like so:

In [ ]:
# We specify the configuration of the model with several set parameters
cfg = {
    "learning_rate": "1e-3",
    "patch_size": "48/256/256",
    "n_samples": 12,
    "vl_interval": 5,
    "n_classes": 2,
    "im_size": "49/256/256",
    "alpha1": 1.0,
    "alpha2": 1.0,
    "loss1": "dice",
    "loss2": "cedice",
    "n_epochs": 10,
    "neg_samples": 1
}

model = SegmentationModel(model_name="dynunet", configuration=cfg)
model.train(data)

Finally, to segment the data, we simply need to call the `predict` method, which returns the corresponding predictions.

In [ ]:
preds = model.predict(data)

### Alternative (directly with a `numpy.array`)

If our data is already loaded, for example, in a `numpy.array`, we can alternatively use the `predict_image` method:

In [ ]:
data = ... # Our data in a `numpy.array` format with (n_volumes, depth, height, width)

preds = model.predict_image(data)